# **Chapter 5.  쇼핑몰 데이터베이스 만들기**


---
### 📝 **학습 목차**
> 1. csv 파일 다루기 - csv <br>
> 2. 데이터 베이스 연결 - sqlight3 <br>
> 3. sqlight 다루기 <br>
> **4. 데이터 백업하기 - iterdump** <br>

#### conn.iterdump


- SQLite 데이터베이스의 스키마 및 데이터를 SQL 스크립트 형식으로 반복적으로 반환하는 메서드<br> 
- 이 메서드는 데이터베이스의 모든 테이블, 뷰, 인덱스 및 해당 내용을 포함하는 SQL 스크립트를 반환. <br>
- `.sql` 파일 확장자로 **테이블을 다시 복원**할 수 있는 쿼리문을 저장 <br>

iterdump() 메서드는 데이터베이스를 통째로 읽고 메모리에 올려야 하기 때문에 큰 데이터베이스의 경우 메모리 사용량이 높을 수 있습니다. 일반적으로 이 메서드는 백업 용도로 사용됩니다

## 4. 데이터 백업하기

In [5]:
import sqlite3

In [6]:
# 연습용 DB 연결
conn = sqlite3.connect("test.db")
# 커서 생성
c = conn.cursor()

In [7]:
# 추가할 상품 리스트
product_list = [[1, '모자', 15000],
                [2, '코트', 200000],
                [3, '티셔츠', 20000],
                [4, '블라우스', 55000],
                [5, '가디건', 45000],
                [6, '청바지', 50000],
                [7, '구두', 150000],
                [8, '가방', 170000]]

In [8]:
# 데이터 여러줄 삽입
c.executemany("INSERT OR REPLACE INTO test(ID, PRODUCT_NAME, PRICE) VALUES(?,?,?)", product_list)
conn.commit()

In [9]:
# iterdump 내용 확인
for line in conn.iterdump():
    print(line)

BEGIN TRANSACTION;
CREATE TABLE test (ID INTEGER PRIMARY KEY, PRODUCT_NAME TEXT, PRICE INTEGER);
INSERT INTO "test" VALUES(1,'모자',15000);
INSERT INTO "test" VALUES(2,'코트',200000);
INSERT INTO "test" VALUES(3,'티셔츠',20000);
INSERT INTO "test" VALUES(4,'블라우스',55000);
INSERT INTO "test" VALUES(5,'가디건',45000);
INSERT INTO "test" VALUES(6,'청바지',50000);
INSERT INTO "test" VALUES(7,'구두',150000);
INSERT INTO "test" VALUES(8,'가방',170000);
COMMIT;


In [10]:
# 데이터 베이스 백업 파일 생성
with conn:
    with open('backup.sql', 'w') as f:
        for line in conn.iterdump():
            f.write('%s\n' % line)
        print('Completed.')

Completed.


In [11]:
# 테이블 삭제
c.execute("DROP TABLE test")
conn.commit()

In [12]:
# 백업 SQL 파일 로딩
with open('backup.sql', 'r') as sql_file:
    sql_script = sql_file.read()

In [13]:
sql_script

'BEGIN TRANSACTION;\nCREATE TABLE test (ID INTEGER PRIMARY KEY, PRODUCT_NAME TEXT, PRICE INTEGER);\nINSERT INTO "test" VALUES(1,\'모자\',15000);\nINSERT INTO "test" VALUES(2,\'코트\',200000);\nINSERT INTO "test" VALUES(3,\'티셔츠\',20000);\nINSERT INTO "test" VALUES(4,\'블라우스\',55000);\nINSERT INTO "test" VALUES(5,\'가디건\',45000);\nINSERT INTO "test" VALUES(6,\'청바지\',50000);\nINSERT INTO "test" VALUES(7,\'구두\',150000);\nINSERT INTO "test" VALUES(8,\'가방\',170000);\nCOMMIT;\n'

In [14]:
# SQL 스크립트 실행
c.executescript(sql_script)
conn.commit()